In [1]:
import os
from pathlib import Path
import datetime as dt

import json
import xarray as xr
import numpy as np
import pandas as pd
from netCDF4 import Dataset as netcdf_dataset

import cmocean

# adding this line to be able to execute script as a cronjob
# see: https://basemaptutorial.readthedocs.io/en/latest/cron.html
import matplotlib as mpl

#mpl.use('Agg')
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from matplotlib.transforms import offset_copy

from math import floor
from scp import SCPClient
from ftplib import FTP

In [2]:
class Settings(object):
    _instance = None
    _fname = "test_settings.json"
    _data = {}

    def set_config(self,_fname):
        with open(_fname) as json_file:
            self._data = json.load(json_file)
        if not os.path.exists(self._data['data_dir']):
            os.makedirs(self._data['data_dir'])
        if not os.path.exists(self._data['out_dir']):
            os.makedirs(self._data['out_dir'])

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(Settings, cls).__new__(cls)
            cls.set_config(cls, cls._fname)
        return cls._instance
    def get(self,atr):
        return self._data[atr]
    def set_attribute(self,atr, value):
        self._data[atr] = value

In [3]:
#file list is []
def get_min_max(wave_dict, file_list):
    wave_min, wave_max = 0, 0
    for f_p in file_list:
        dataset = netcdf_dataset(os.path.join(Settings().get('data_dir'), f_p))
        for wave in wave_dict.keys():
            wave_data = dataset.variables[wave][0, :, :]
            #wtf why i have temp here?
            temp_min = wave_data.min()
            temp_max = wave_data.max()
            if temp_min < wave_min:
                wave_min = temp_min
            if temp_max > wave_max:
                wave_max = temp_max
        dataset.close()
    return (wave_min, wave_max)


In [4]:
def make_ticks(w_min, w_max):
    scale = w_max - w_min
    if scale > 10:
        num = floor((w_max - w_min)/2) + 1
    else:
        num = floor(w_max) + 1
    ticks = np.linspace(floor(w_min), floor(w_max), num=num, dtype = int)
    labels = []
    for t in ticks:
        if t == 0:
            labels.append('flat')
        elif t == 1:
            labels.append('1 meter')
        else:
            labels.append(str(t)+ ' meters')
    return ticks, labels

In [8]:
wave_dict = {"VHM0_WW": "significant wind wave height",
            "VHM0_SW1": "significant primary swell wave height",
            "VHM0_SW2": "significant secondary swell wave height"}

stgs = Settings()
proj = ccrs.PlateCarree()
#settings_file = '
data_file = 'testfile1.nc'
wave_type = "VHM0_WW"
wave_dict = {wave_type : "whatever"}
print(wave_dict)

file_path = os.path.join(Settings().get('data_dir'),data_file)
data_list = [file_path] 
dataset = netcdf_dataset(file_path)
dataset_xr = xr.open_dataset(file_path)
(w_min, w_max) = get_min_max(wave_dict, data_list)
wave = {"var": wave_type, "name": "significant wind wave height", "min": w_min, "max": w_max}
lats = dataset.variables['latitude'][:]
lons = dataset.variables['longitude'][:]
wave_tn = dataset.variables[wave_type][0, :, :]
cmap = cmocean.cm.deep
#cmap = cmocean.cm.tempo
levels = MaxNLocator(nbins=10).tick_values(w_min, w_max)
norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
ticks,labels = make_ticks(w_min, w_max)
  
wave_var = wave_type
wave_name = "trala"
#    wave_var = wave['var']
 #   wave_name = wave['name']
dx, dy = 0.05, 0.05

{'VHM0_WW': 'whatever'}


In [9]:
def jupyter_single_frame_ultra( i_time):
    
    frame_n = 1
    time_stamp_n = time_string
    
    wave_tn = dataset.variables[wave_var][i_time, :, :]
    fig = plt.figure(figsize=(12, 6), clear=False)
    
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
   # bg = fig.canvas.copy_from_bbox(fig.bbox)
    
    #fig.canvas.blit(fig.bbox)
    #tutaj
   # ax.clear() 
    ax.set_extent([stgs.get('min_lon'),stgs.get('max_lon'),stgs.get('min_lat'),stgs.get('max_lat')], crs=proj)
    geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
    text_transform = offset_copy(geodetic_transform, units='dots', x=-25)

    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic())

    ax.text(stgs.get('text_lon'), stgs.get('text_lat'), stgs.get('text_label'),
            verticalalignment='bottom', horizontalalignment='left',
            transform=text_transform,
            bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))

    
    ax.set_title(wave_name + " " + time_stamp_n, fontsize=16)
    
    gr= ax.gridlines(draw_labels = True)
    gr.right_labels = False
    ax.coastlines()
    ax.plot(stgs.get('point_lon'), stgs.get('point_lat'), marker='o', color='red', markersize=8,
            alpha=0.7, transform=ccrs.Geodetic(), animated=True)
    cs = plt.contour(lons, lats, wave_tn, levels = levels, colors='w', transform=ccrs.PlateCarree())
    ax.clabel(cs, colors='w', fontsize=20)
    contour_img = ax.contourf(lons + dx/2., lats + dy/2.,wave_tn, levels=levels, cmap=cmap)
    cbar = fig.colorbar(contour_img, ax=ax, ticks = ticks)
    cbar.ax.set_yticklabels(labels)
    
    plt.show(block=False)
   

In [10]:
import ipywidgets as widgets
%matplotlib inline
time_xr = dataset_xr['time']
t0 = time_xr[0]
ts = pd.to_datetime(str(t0.data)) 
time_string = ts.strftime('%Y.%m.%d %Hh:%M')

#----------------

   

interactive_plot = widgets.interactive(jupyter_single_frame_ultra,  i_time = widgets.IntSlider(min=1, max=10, step=1, value=1))
output = interactive_plot.children[-1]
output.layout.height = '600px'
output.layout.weight = '500px'
interactive_plot

interactive(children=(IntSlider(value=1, description='i_time', max=10, min=1), Output(layout=Layout(height='60…

In [11]:
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…